In [1]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

from GraphAPI import GraphCreator
from graph_helpers import *
from evaluations import *

%aimport GraphAPI
%aimport graph_helpers
%aimport evaluations

## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [105]:
gc = GraphCreator("Scikit-learn")
print("Number of Links to Search:", len(gc.next_links))
print(gc.primary_nodes)

Number of Links to Search: 104
{'Mlpy': True, 'Matplotlib': True, 'TensorFlow': True, 'List of numerical analysis software': True, 'Natural Language Toolkit': True, 'SpaCy': True, 'NumPy': True, 'Orange (software)': True}


2. We query all the nodes linked to/from the entry point (expand our network one level for each node).

In [106]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph and ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect node. 

In [107]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [108]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,SciPy,NumPy,8
1,NumPy,SciPy,8
2,ML.NET,TensorFlow,7
3,C (programming language),C++,5
4,Python (programming language),C++,5


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [109]:
features_df = gc.get_features_df(rank=False)

In [110]:
features_df.sort_values("degree", ascending=False)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,primary_link
0,Linux,10922,0,9864.0,1058.0,0.002494,2.107965e-02,0.067871,1112.0,1.0,2.0,0
1,Software categories,10058,0,9960.0,98.0,0.001976,1.657237e-02,0.048523,0.0,1.0,3.0,0
2,MacOS,6041,0,5155.0,886.0,0.003777,1.632806e-02,0.039084,1088.0,1.0,3.0,0
3,C++,5440,0,4698.0,742.0,0.005074,5.577028e-02,0.035709,1144.0,1.0,3.0,0
4,C (programming language),5221,0,4723.0,498.0,0.004417,4.600668e-02,0.036973,604.0,1.0,3.0,0
5,Python (programming language),3608,0,3071.0,537.0,0.014147,6.787157e-02,0.019381,502.0,1.0,1.0,0
6,Free software,3584,0,3197.0,387.0,0.005569,2.150534e-02,0.032532,474.0,1.0,2.0,0
7,Repository (version control),2943,0,2872.0,71.0,0.004722,9.393394e-03,0.011239,96.0,1.0,3.0,0
8,Machine learning,2609,0,2081.0,528.0,0.018326,1.619801e-01,0.015960,496.0,1.0,1.0,0
9,Regression analysis,1849,0,1201.0,648.0,0.018414,2.371553e-01,0.007925,990.0,1.0,2.0,0


## Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [115]:
gc.rank_similarity()

print(gc.see_also_articles)

gc.features_df[[
    "node", 
    "category_matches_with_source", 
    "primary_link", 
    "shared_neighbors_with_entry_score",
    "shortest_path_length_from_entry",
    "shortest_path_length_to_entry",
    "centrality",
    "similarity_rank", 
]].sort_values(["similarity_rank", "centrality"], ascending=False).reset_index().drop("index", axis=1).head(50)
# gc.features_df[gc.features_df.category_matches_with_source == 1]

['Mlpy', 'SpaCy', 'Natural Language Toolkit', 'Orange (software)', 'TensorFlow', 'List of numerical analysis software', 'NumPy', 'Matplotlib']


,node,category_matches_with_source,primary_link,shared_neighbors_with_entry_score,shortest_path_length_from_entry,shortest_path_length_to_entry,centrality,similarity_rank
0,Scikit-learn,1,0,1.000000,0.0,0.0,0.052251,inf
1,TensorFlow,4,1,0.030111,2.0,2.0,0.019039,2.515055
2,Pandas (software),3,0,0.212963,2.0,2.0,0.010234,1.606481
3,LIBSVM,3,0,0.200000,2.0,2.0,0.011920,1.600000
4,Matplotlib,2,1,0.124138,2.0,2.0,0.010641,1.562069
5,NumPy,2,1,0.117021,2.0,2.0,0.012128,1.558511
6,Orange (software),0,1,0.270588,1.0,1.0,0.023044,1.270588
7,Mlpy,0,1,0.176471,1.0,1.0,0.002686,1.176471
8,Scikit-image,3,0,0.266667,4.0,2.0,0.006704,1.088889
9,SpaCy,0,1,0.082353,1.0,1.0,0.000978,1.082353


# Evaluation

In [112]:
evaluate_recommendations(gc.features_df, on="similarity_rank", targets=gc.see_also_articles)

,value
description,
score,0.999622
% targets in top 1%,0.875000
% targets in top 5%,0.875000
% targets in top 10%,0.875000
% targets in top 20%,0.875000
max score posible,0.999748
difference,0.000126
total targets,8.000000


# Basic Plotting

In [ ]:
sns.pairplot(features_df)

In [ ]:
sns.heatmap(features_df.corr())

# Intro Node Values

In [ ]:
intro_nodes = gc.intro_nodes
intro_nodes_df = None
for node in intro_nodes:
    if df is None:
        intro_nodes_df = pd.DataFrame(features_df[features_df.node == node])
    else:
        intro_nodes_df = pd.concat([intro_nodes_df, pd.DataFrame(features_df[features_df.node == node])])
    
intro_nodes_df

In [ ]:
sns.pairplot(intro_nodes_df, hue="node")